In [14]:
pip install tensorflow[and-cuda] numpy h5py matplotlib tqdm scipy reservoirpy keras pandas scikit-learn

INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.8 MB 3.3 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/14.8 MB 3.7 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/14.8 MB 3.4 MB/s eta 0:00:04
   ------- -------------------------------- 2.6/14.8 MB 3.4 MB/s eta 0:00:04
   --------- ------------------------------ 3.4/14.8 MB 3.4 MB/s eta 0:00:04
   ---------- ----------------------------- 3.9/14.8 MB 3.2 MB/s eta 0:00:04
   ------------ --------------------------- 4.7/14.8 MB 3.3 MB/s eta 0:00:04
   -------------- ------------------------- 5.2/14.8 MB 3.2 MB/s eta 0:00:03
   ---------------- ----------------------- 6.0/14.8 MB 3.3 MB/s eta 0:00:03
   ------------------ --------------------- 6.8/14.8 MB 3.4 MB/s eta 0:00:03
   ---------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
classiq 0.70.0 requires networkx<3.0.0,>=2.5.1, but you have networkx 3.2.1 which is incompatible.
qiskit-ibmq-provider 0.20.2 requires numpy<1.24, but you have numpy 1.24.3 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: C:\Users\gedam\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import reservoirpy as rpy
import h5py
import numpy as np
import sklearn
ascad_path = r"F:\QRC-SCA-Project\ASCAD_data\ASCAD_data\ASCAD_databases\ASCAD.h5"

In [3]:
with h5py.File(ascad_path, "r") as f:
    # Profiling (training) set
    X_profiling = np.array(f["Profiling_traces/traces"])
    Y_profiling = np.array(f["Profiling_traces/labels"])

    # Attack (test) set
    X_attack = np.array(f["Attack_traces/traces"])
    Y_attack = np.array(f["Attack_traces/labels"])

print(X_profiling.shape, Y_profiling.shape)
print(X_attack.shape, Y_attack.shape)

(50000, 700) (50000,)
(10000, 700) (10000,)


In [9]:
import numpy as np
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

# --- Project Constants ---
FILE_PATH = r"F:\QRC-SCA-Project\ASCAD_data\ASCAD_data\ASCAD_databases\ASCAD.h5" 
WINDOW_SIZE = 4            # D_in = 4 (for 4 qubits)
STRIDE = 1                 
QUANTUM_RANGE_MAX = 2 * np.pi 
TARGET_HW = 4              # Target Hamming Weight for binary detection 

def load_ascad_data(file_path):
    """Loads traces and labels using the correct ASCAD HDF5 structure."""
    try:
        with h5py.File(file_path, 'r') as f:
            # CORRECTED PATHS based on your working code
            X_prof = np.array(f['Profiling_traces/traces']) 
            Y_prof = np.array(f['Profiling_traces/labels']) 

            X_att = np.array(f['Attack_traces/traces'])
            Y_att = np.array(f['Attack_traces/labels'])
            
            print(f"Data loaded successfully!")
            print(f"Profiling Traces: {X_prof.shape}, Attack Traces: {X_att.shape}")
            return X_prof, Y_prof, X_att, Y_att
    except Exception as e:
        print(f"CRITICAL ERROR loading HDF5 file: {e}")
        # Terminate if data loading fails
        raise SystemExit(e)

def to_binary_hw_detection(Y_raw, target_hw):
    """Converts 256-class S-Box labels to binary detection labels (0 or 1)."""
    # 1. Compute Hamming Weight (HW) of S-box output (0 to 255)
    HW = np.array([bin(n).count("1") for n in range(0, 256)])
    Y_hw = HW[Y_raw]
    
    # 2. Binary Label: 1 if HW == target_hw (Attack), 0 otherwise (Normal)
    Y_binary = (Y_hw == target_hw).astype(int)
    print(f"  -> Attack (HW={target_hw}) traces count: {np.sum(Y_binary)}")
    return Y_binary

def window_and_normalize(X_traces, Y_labels, window_size, stride, min_val, max_val):
    """Performs time-series windowing and then normalizes features."""
    windows = []
    window_labels = []
    
    for trace, label in zip(X_traces, Y_labels):
        for i in range(0, len(trace) - window_size + 1, stride):
            windows.append(trace[i:i + window_size])
            window_labels.append(label)

    U = np.array(windows, dtype=np.float32)
    Y = np.array(window_labels, dtype=np.int32)
    
    # Normalization: U_normalized = (U - Min) / (Max - Min) * QUANTUM_RANGE_MAX
    U_normalized = (U - min_val) / (max_val - min_val) * QUANTUM_RANGE_MAX
    
    return U_normalized, Y

# --- Main Data Execution ---
# --- Main Data Execution (REVISED) ---
# ... (rest of the code remains the same)

X_prof_raw, Y_prof_raw, X_att_raw, Y_att_raw = load_ascad_data(FILE_PATH)

# NEW: CRITICAL DOWN-SAMPLING STEP
N_TRAIN_TRACES_POC = 1000  # Use only the first 1,000 profiling traces
N_TEST_TRACES_POC = 1000   # Use only the first 1,000 attack traces for a balanced PoC test

X_prof_poc = X_prof_raw[:N_TRAIN_TRACES_POC]
Y_prof_poc = Y_prof_raw[:N_TRAIN_TRACES_POC]
X_att_poc = X_att_raw[:N_TEST_TRACES_POC]
Y_att_poc = Y_att_raw[:N_TEST_TRACES_POC]

# Global Min/Max calculation (should be based on the POC data now)
GLOBAL_MIN = min(X_prof_poc.min(), X_att_poc.min())
GLOBAL_MAX = max(X_prof_poc.max(), X_att_poc.max())

# 1.5 Binary Label Simplification
Y_prof_bin = to_binary_hw_detection(Y_prof_poc, TARGET_HW)
Y_att_bin = to_binary_hw_detection(Y_att_poc, TARGET_HW)

# 1.7 & 1.8 Windowing and Normalization
U_train, Y_train = window_and_normalize(X_prof_poc, Y_prof_bin, WINDOW_SIZE, STRIDE, GLOBAL_MIN, GLOBAL_MAX)
U_test, Y_test = window_and_normalize(X_att_poc, Y_att_bin, WINDOW_SIZE, STRIDE, GLOBAL_MIN, GLOBAL_MAX)

# ... (rest of the save/print block remains the same)
print(f"NEW U_train shape (approx 697k steps): {U_train.shape}") # Verify the size is now small

print("\n--- Final Data Structure ---")
print(f"Training Input U_train shape: {U_train.shape}")
print(f"Test Input U_test shape: {U_test.shape}")

# Save the prepared data for use in the next scripts
np.save('U_train.npy', U_train)
np.save('Y_train.npy', Y_train)
np.save('U_test.npy', U_test)
np.save('Y_test.npy', Y_test)
print("Data saved to .npy files for Stage 2 & 3.")

Data loaded successfully!
Profiling Traces: (50000, 700), Attack Traces: (10000, 700)
  -> Attack (HW=4) traces count: 239
  -> Attack (HW=4) traces count: 260
NEW U_train shape (approx 697k steps): (697000, 4)

--- Final Data Structure ---
Training Input U_train shape: (697000, 4)
Test Input U_test shape: (697000, 4)
Data saved to .npy files for Stage 2 & 3.


In [11]:
import numpy as np
from sklearn.linear_model import RidgeClassifier 
from sklearn.metrics import accuracy_score, f1_score
import warnings
# Suppress warnings that might clutter the output
warnings.filterwarnings("ignore")

# --- Load Data (Data should be loaded from your .npy files) ---
try:
    U_train = np.load('U_train.npy')
    Y_train = np.load('Y_train.npy')
    U_test = np.load('U_test.npy')
    Y_test = np.load('Y_test.npy')
except FileNotFoundError:
    print("CRITICAL ERROR: Data files not found. Please run data_prep.py first.")
    raise SystemExit()

# --- ESN Hyperparameters ---
N_x = 100 # Reservoir size
D_in = 4 # Input dimension
SPECTRAL_RADIUS = 0.95 
LEAKING_RATE = 0.5       
RIDGE_REGULARIZATION = 1e-7

# Set seed for reproducible random weights
np.random.seed(42)

# --- 1. Weight Initialization (The Fixed Reservoir) ---
print("1/3: Initializing fixed ESN weight matrices...")


# 1.1 Input Weight Matrix (W_in): Maps D_in input to N_x reservoir units
W_in = np.random.uniform(-0.1, 0.1, (N_x, D_in)) 

# 1.2 Internal Weight Matrix (W): Maps N_x units to N_x units
W = np.random.uniform(-1, 1, (N_x, N_x))

# 1.3 Apply Spectral Radius Scaling
eigenvalues = np.linalg.eigvals(W)
current_sr = np.max(np.abs(eigenvalues))
W = W * (SPECTRAL_RADIUS / current_sr)

# --- 2. State Collection (Manual Run) ---
def run_manual_reservoir(U_data, W_in, W, N_x, lr):
    """Evolves the reservoir state x(t) for the given input sequence U_data."""
    N_steps = U_data.shape[0]
    R_cl = np.zeros((N_steps, N_x)) # Matrix to store all collected states
    x = np.zeros(N_x)              # Initial state vector x(0)
    
    for t in range(N_steps):
        u_t = U_data[t]
        
        # ESN State Update Formula (Vectorized)
        pre_activation = W @ x + W_in @ u_t
        x_new = (1 - lr) * x + lr * np.tanh(pre_activation)
        
        # Store the new state vector and update the current state
        R_cl[t] = x_new
        x = x_new 

    return R_cl

# Collect states for training and testing
print("2/3: Collecting Classical Reservoir States (Manual Run)...")
R_Cl_train = run_manual_reservoir(U_train, W_in, W, N_x, LEAKING_RATE) 
R_Cl_test = run_manual_reservoir(U_test, W_in, W, N_x, LEAKING_RATE) 

# --- 3. Train Readout and Evaluate (FIXED) ---
print("3/3: Training Scikit-learn Ridge Readout (with balanced class weights)...")

# FIX: Use class_weight='balanced' to prioritize the minority "Attack" class (1)
readout_clf = RidgeClassifier(alpha=RIDGE_REGULARIZATION, class_weight='balanced')
readout_clf.fit(R_Cl_train, Y_train)

# Evaluation
Y_pred_cl = readout_clf.predict(R_Cl_test)

# --- Calculate Metrics ---
accuracy_cl = accuracy_score(Y_test, Y_pred_cl)
f1_cl = f1_score(Y_test, Y_pred_cl) 

print("\n--- Classical ESN Benchmark Results (Manual Implementation) ---")
print(f"Accuracy: {accuracy_cl:.4f}")
print(f"F1-Score (A_Cl): {f1_cl:.4f}")
print("---------------------------------------")

# Save the benchmark score
np.save('A_Cl_benchmark.npy', f1_cl)

print("Classical baseline FIXED. Move to Stage 3 (Quantum QRC).")

1/3: Initializing fixed ESN weight matrices...
2/3: Collecting Classical Reservoir States (Manual Run)...
3/3: Training Scikit-learn Ridge Readout (with balanced class weights)...

--- Classical ESN Benchmark Results (Manual Implementation) ---
Accuracy: 0.5006
F1-Score (A_Cl): 0.3426
---------------------------------------
Classical baseline FIXED. Move to Stage 3 (Quantum QRC).
